In [1]:
! pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.2 MB/s eta 0:00:00


In [3]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np

import warnings
warnings.filterwarnings('ignore')

MATCH_SCORE = 5
MISMATCH_SCORE = -4
GAP_SCORE = -2
GAP_EXTENSION_SCORE = -1

def needleman_wunsch(seq1, seq2):
    n = len(seq1)
    m = len(seq2)
    alignment_matrix = np.zeros((n+1, m+1), dtype=int)
    traceback_matrix = np.zeros((n+1, m+1), dtype=int)
    
    for i in range(1, n+1):
        alignment_matrix[i][0] = GAP_SCORE * i
        traceback_matrix[i][0] = 1
    
    for j in range(1, m+1):
        alignment_matrix[0][j] = GAP_SCORE * j
        traceback_matrix[0][j] = 2
        
    for i in range(1, n+1):
        for j in range(1, m+1):
            match_score = MATCH_SCORE if seq1[i-1] == seq2[j-1] else MISMATCH_SCORE
            diagonal = alignment_matrix[i-1][j-1] + match_score
            up = alignment_matrix[i-1][j] + GAP_SCORE
            left = alignment_matrix[i][j-1] + GAP_SCORE
            max_score = max(diagonal, up, left)
            alignment_matrix[i][j] = max_score
            if max_score == diagonal:
                traceback_matrix[i][j] = 0
            elif max_score == up:
                traceback_matrix[i][j] = 1
            else:
                traceback_matrix[i][j] = 2
            
    print("Alignment Matrix:")
    for row in alignment_matrix:
        print(row)
    
    print("Traceback Matrix:")
    for row in traceback_matrix:
        print(row)
    
    alignments = pairwise2.align.globalms(seq1, seq2, MATCH_SCORE, MISMATCH_SCORE, GAP_SCORE, GAP_EXTENSION_SCORE)
    for a in alignments:
        print(format_alignment(*a))
        
    aligned_seq1 = ''
    aligned_seq2 = ''
    i = n
    j = m

    while i > 0 or j > 0:
        direction = traceback_matrix[i][j]
        if direction == 0:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif direction == 1:
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1
    
    print("Aligned Sequences:")
    print(aligned_seq1)
    print(aligned_seq2)


seq1 = "AAAACCCCGGGGTTTT"
seq2 = "TAAAACCCCGGGGTTT"
needleman_wunsch(seq1, seq2)

Alignment Matrix:
[  0  -2  -4  -6  -8 -10 -12 -14 -16 -18 -20 -22 -24 -26 -28 -30 -32]
[ -2  -4   3   1  -1  -3  -5  -7  -9 -11 -13 -15 -17 -19 -21 -23 -25]
[ -4  -6   1   8   6   4   2   0  -2  -4  -6  -8 -10 -12 -14 -16 -18]
[ -6  -8  -1   6  13  11   9   7   5   3   1  -1  -3  -5  -7  -9 -11]
[ -8 -10  -3   4  11  18  16  14  12  10   8   6   4   2   0  -2  -4]
[-10 -12  -5   2   9  16  23  21  19  17  15  13  11   9   7   5   3]
[-12 -14  -7   0   7  14  21  28  26  24  22  20  18  16  14  12  10]
[-14 -16  -9  -2   5  12  19  26  33  31  29  27  25  23  21  19  17]
[-16 -18 -11  -4   3  10  17  24  31  38  36  34  32  30  28  26  24]
[-18 -20 -13  -6   1   8  15  22  29  36  43  41  39  37  35  33  31]
[-20 -22 -15  -8  -1   6  13  20  27  34  41  48  46  44  42  40  38]
[-22 -24 -17 -10  -3   4  11  18  25  32  39  46  53  51  49  47  45]
[-24 -26 -19 -12  -5   2   9  16  23  30  37  44  51  58  56  54  52]
[-26 -19 -21 -14  -7   0   7  14  21  28  35  42  49  56  63  61  59]
[-